<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Graphlit_Python_SDK_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


Initialize Graphlit

In [3]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [17]:
import asyncio
import time
from datetime import datetime

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.delete_content(content_id)

        return response.delete_content.id if response.delete_content is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Completion",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(model=enums.OpenAIModels.GPT4O_128K),
        searchType=enums.ConversationSearchTypes.VECTOR,
        numberSimilar=25,
        systemPrompt="Answer concisely as one item, containing one paragraph.",
        promptStrategy=input_types.PromptStrategyInput(
            type=enums.PromptStrategyTypes.OPTIMIZE_SEARCH
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION,
            contentLimit=10,
        ),
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(name="Conversation", specification=input_types.EntityReferenceInput(id=specification_id))

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientHttpError as e:
        print(str(e))
        return None

async def delete_specification(specification_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.delete_specification(specification_id)

        return response.delete_specification.id if response.delete_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.delete_conversation(conversation_id)

        return response.delete_conversation.id if response.delete_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

Define Graphlit RAG function

In [18]:
async def prompt_conversation(prompt: str, conversation_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        message = response.prompt_conversation.message if response.prompt_conversation is not None else None

        return message
    except exceptions.GraphQLClientHttpError as e:
        print(str(e))
    finally:
        _ = await graphlit.client.delete_conversation(conversation_id)

Execute Graphlit example

In [24]:
specification_id = await create_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id)

    if conversation_id is not None:
        print(f'Created conversation [{conversation_id}].')

        content_id = await ingest_uri(uri="https://www.graphlit.com")

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            prompt = "How can Graphlit accelerate my Generative AI app development?"

            print(f'User: {prompt}')

            message = await prompt_conversation(prompt, conversation_id)

            print(f'Assistant: {message.message.strip() if message is not None else None}')

            await delete_content(content_id)
        else:
            print('Failed to ingest content.')

        await delete_conversation(conversation_id)
    else:
        print('Failed to create conversation.')

    await delete_specification(specification_id)
else:
    print('Failed to create specification.')


Created specification [3a440f13-93a6-4b79-8fa9-df29ae72d973].
Created conversation [13319df8-210b-4d04-b6a8-1594f7d668c9].
Ingested content [0f6b5cdc-407d-43af-bb39-c5c65160922f].
User: How can Graphlit accelerate my Generative AI app development?
Assistant: Graphlit accelerates Generative AI app development by providing an API-first platform that integrates unstructured data with large language models (LLMs) in minutes. It supports ingestion from various data sources and formats, including documents, audio, video, and images. The platform offers automated ETL workflows, high-quality OCR text extraction, and built-in vector embeddings for semantic search and RAG conversations. Additionally, Graphlit is serverless, cloud-native, and requires no infrastructure deployment, making it easy to start building AI-powered applications immediately.
